In [9]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelWithLMHead, LineByLineTextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, TrainerCallback,ProgressCallback
import torch
import random

In [2]:
# Carregar o modelo e o tokenizador
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")
model = AutoModelWithLMHead.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Definir o comprimento máximo de sequência para 1024 tokens
tokenizer.model_max_length = 1024


/home/rafaelrosendo/anaconda3/envs/torch2/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [3]:
# Carregar seus dados do CSV (substitua o valor de 'path_to_csv' pelo caminho do seu arquivo CSV)
path_to_csv = "/home/rafaelrosendo/gpt2/portuguese-poems.csv"
data = pd.read_csv(path_to_csv)["Content"].tolist()

In [4]:
# Embaralhar a lista de dados para garantir uma divisão aleatória
random.shuffle(data)

# Definir a proporção do conjunto de validação
validation_ratio = 0.2

# Calcular o índice para dividir os dados
split_index = int(len(data) * (1 - validation_ratio))

# Dividir os dados em treinamento e validação
train_data = data[:split_index]
validation_data = data[split_index:]

In [5]:
# Converter os dados em um formato adequado para o treinamento do modelo
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path_to_csv,
    block_size=128,
)
validation_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path_to_csv,
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)



/home/rafaelrosendo/anaconda3/envs/torch2/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [21]:
# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./gpt2_fine_tuned",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [ ]:
# Inicializar o Trainer e iniciar o treinamento
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    callbacks=[ProgressCallback()]
)

trainer.train()

In [24]:


def generate_text(model, tokenizer, input_text, max_length=50, top_k=40):
    inputs = tokenizer(input_text, return_tensors="pt")
    sample_outputs = model.generate(inputs.input_ids,
                                    pad_token_id=tokenizer.eos_token_id,
                                    do_sample=True, 
                                    max_length=max_length,
                                    top_k=top_k,
                                    num_return_sequences=1)

    generated_text = tokenizer.decode(sample_outputs[0].tolist())
    return generated_text

# Carregar o modelo e o tokenizador
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")
model = AutoModelWithLMHead.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Definir o comprimento máximo de sequência para 1024 tokens
tokenizer.model_max_length = 1024

# input sequence
text = "O mar salgado quanto de teu sal não são lágrimas de Portugal"
generated_text = generate_text(model, tokenizer, text, max_length=150, top_k=40)
print("Generated text:")
print(generated_text)

Generated text:
O mar salgado quanto de teu sal não são lágrimas de Portugal, mas lágrimas de um povo que tem vivido em paz. A lágrimas não podem mais vir a falar para mim, nem me ser capaz de ser ouvida. Os portugueses vão falar para mim, eu orei e ele tenho o direito de falar em Portugal para mim."

"Não sei se eu terei de ser o próximo rei. Eu terei de um rei não nascido fora do mundo, não por Deus".

"Vocês sabem que foi meu soberano. É difícil conseguir me fazer um rei que não seja de minha consciência e sem o apoio de Deus"

"A grande esperança da minha pessoa é que ele me dê a tudo que ele, que é necessário
